In [3]:
import pandas as pd
import datetime
import sqlite3

#---------Loading data using sqlite---------------
conn = sqlite3.connect("sales.db")
query = """
SELECT 
    OrderDate, 
    Sales, 
    CustomerID, 
    OrderID
FROM 
    sales
"""
df = pd.read_sql(query, conn)

#---------Convert OrderDate to datetime-----------
df['OrderDate'] = pd.to_datetime(df['OrderDate'], dayfirst = True)

In [4]:
import datetime as dt

#-------Define the snapshot date (the latest purchase date in the dataset)----
snapshot_date = df['OrderDate'].max() + dt.timedelta(days=1)

#--------Calculate RFM metrics------------------------------------------------
rfm = df.groupby('CustomerID').agg({
    'OrderDate': lambda x: (snapshot_date - x.max()).days,           # Recency of purchase
    'OrderID': 'count',                                              # Frequency of purchase
    'Sales': 'sum'                                                   # Amount spent (Monetary) 
})

#--------Rename columns to relevent names--------------------------------------
rfm.columns = ['Recency', 'Frequency', 'Monetary']

#--------Assign RFM scores (1-5) based on analysis-----------------------------
rfm['R_Score'] = pd.qcut(rfm['Recency'], 5, labels=[5, 4, 3, 2, 1])
rfm['F_Score'] = pd.qcut(rfm['Frequency'].rank(method="first"), 5, labels=[1, 2, 3, 4, 5])
rfm['M_Score'] = pd.qcut(rfm['Monetary'], 5, labels=[1, 2, 3, 4, 5])

#--------Create RFM segment by combining scores---------------------------------
rfm['RFM_Segment'] = rfm['R_Score'].astype(str) + rfm['F_Score'].astype(str) + rfm['M_Score'].astype(str)

print(rfm.head(5))


            Recency  Frequency  Monetary R_Score F_Score M_Score RFM_Segment
CustomerID                                                                  
AA-10315        185         11  5563.560       2       3       5         235
AA-10375         20         15  1056.390       5       4       2         542
AA-10480        260         12  1790.512       1       3       3         133
AA-10645         56         18  5086.935       3       5       5         355
AB-10015        416          6   886.156       1       1       1         111
